In [35]:
checkins = sc.textFile('../YelpDataset/yelp_academic_dataset_checkin.json')
businesses = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')

In [38]:
import operator
import math
import json

selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']

def mapper_checkins_filter(records):
    for line in records:
        line = json.loads(line)
        checkin_info = line['checkin_info']
        for key, val in checkin_info.iteritems():
            yield (line['business_id'], val)

def mapper_business_filter(rows):
    for row in rows:
        row = json.loads(row)
        for category in row['categories']:
            if(category in selectedCategories):
                yield (row['business_id'], category)

def mapper_normalization(records):
    for category in records:
        minCheckins = min(category[1], key=operator.itemgetter(1))[1]
        maxCheckins = max(category[1], key=operator.itemgetter(1))[1]
        for business in category[1]:
            normalizedRatingValue = (float(business[1]) - minCheckins)/(maxCheckins - minCheckins) * 20
            yield ((category[0], business[0]), normalizedRatingValue)    
        
checkins_filtered = checkins.mapPartitions(mapper_checkins_filter).reduceByKey(operator.add)
business_filtered = businesses.mapPartitions(mapper_buisness_filter)
joined = checkins_filtered.join(business_filtered).map(lambda x: (x[1][1],(x[0],x[1][0]))).groupByKey().mapValues(list)
total_normalized = joined.mapPartitions(mapper_normalization)
# total_sorted = total_normalized.map(lambda (k, v): (k, sorted(v, key=lambda x: x[1], reverse=True)))
total_normalized.take(20)

[((u'Coffee & Tea', u'sOcZcXcNm8LmdoOYqEDqpg'), 0.021596328624133895),
 ((u'Coffee & Tea', u'4K7nR-e9staGTNJk1ez7MA'), 0.039593269144245484),
 ((u'Coffee & Tea', u'ycutyiy4ozAX1t2wCIeP2w'), 0.04319265724826779),
 ((u'Coffee & Tea', u'OYZOzzipfXdWrYe1A2zeKw'), 0.025195716728156212),
 ((u'Coffee & Tea', u'0EXVIF07vw-QTTy9V7-ebg'), 0.18356879330513814),
 ((u'Coffee & Tea', u'OG27njKJGhjpesJb9McGPQ'), 0.10618194906865833),
 ((u'Coffee & Tea', u'qAkk0DaKLotxX2UzSPg9fQ'), 0.017996940520111582),
 ((u'Coffee & Tea', u'ziA1FCZrt9SMjlBlPo5EaQ'), 0.35813911635022044),
 ((u'Coffee & Tea', u'JJGBx48I4yudiEhw9fGbZw'), 0.026995410780167373),
 ((u'Coffee & Tea', u'sB0yyxeBcVhRxGd-lwR4yg'), 0.01619724646810042),
 ((u'Coffee & Tea', u'toDZ6L_a_0lWU9O1JtQRGg'), 0.31854584720597495),
 ((u'Coffee & Tea', u'SOtYTZQIB5-3EqIlFjo7lA'), 0.03059479888418969),
 ((u'Coffee & Tea', u'y3VxXuUvrhURPbvtF0GkTg'), 0.20876451003329435),
 ((u'Coffee & Tea', u'PkPcOZh1tQfQUV6bJRTrng'), 0.2267614505534059),
 ((u'Coffee & Te